In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing external packages
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Importing internal packages
from src.features.point_cloud_handler import Handler
from src.features.pre_process import Reduce
from src.enums import Files
from src.features.slice import Slice
from src.features.display import Visualize

In [4]:
raw_point_cloud: o3d.geometry.PointCloud = Handler.open(Files.TREE_POINT_CLOUD)

Loaded 17448215 points from tree_point_cloud.las


In [5]:
down_sampled_point_cloud: o3d.geometry.PointCloud = Reduce.voxel_downsample(
    point_cloud=raw_point_cloud
)

Original point count: 17448215
Downsampled point count: 6183892
Point cloud size reduced with 64.56%


In [6]:
slice: Slice = Slice(point_cloud=down_sampled_point_cloud)

(4,)
(5,)
(6,)


In [7]:
for s in slice.slices:
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(s)
    Visualize.display(point_cloud)
    